# Haar System

In [7]:
import numpy as np

def approximation(array):
    return array[:array.shape[0]//2]

def details(array):
    return array[array.shape[0]//2:]

root2 = np.sqrt(2)
s = np.array([32, 32, 16, 8, 24, 16, 64, 32])

# 1 dimension

## Decomposition 

In [8]:
def direct_wavelet(array,output):
    N, N2, = array.shape[0], array.shape[0] // 2
    for i in range(0,N,2):
        output[int(i/2)] = (array[i] + array[i+1])/root2
        output[int(i/2 + N2)] = (array[i] - array[i+1])/root2
    return output

def inverse_wavelet(output,signal):
    N, N2,  = output.shape[0], output.shape[0] // 2
    for i in range(0,N,2):
        signal[i] = (output[int(i/2)] + output[int(i/2 + N2)])/root2
        signal[i+1] = (output[int(i/2)] - output[int(i/2 + N2)])/root2
    return signal


In [9]:
# DIRECT WAVELET TRANSFORM --> dwt
dwt = np.empty(len(s))
dwt = direct_wavelet(s,dwt)
print("\nApproximation coefficients (A1):", approximation(dwt))
print("Detail coefficients (D1):", details(dwt))

dwt2 = np.empty(len(approximation(dwt)))
dwt2 = direct_wavelet(approximation(dwt),dwt2)
print("\nApproximation coefficients (A2):", approximation(dwt2))
print("Detail coefficients (D2):", details(dwt2))


dwt3 = np.empty(len(approximation(dwt2)))
dwt3 = direct_wavelet(approximation(dwt2),dwt3)
print("\nApproximation coefficients (A3):", approximation(dwt3))
print("Detail coefficients (D3):", details(dwt3))


Approximation coefficients (A1): [45.254834   16.97056275 28.28427125 67.88225099]
Detail coefficients (D1): [ 0.          5.65685425  5.65685425 22.627417  ]

Approximation coefficients (A2): [44. 68.]
Detail coefficients (D2): [ 20. -28.]

Approximation coefficients (A3): [79.19595949]
Detail coefficients (D3): [-16.97056275]


## Reconstruction

In [10]:
# INVERSE WAVELET TRANSFORM
print("\nApproximation coefficients (A3):", approximation(dwt3))

rest_s2 = np.empty(2*len(approximation(dwt3)))
rest_s2 = inverse_wavelet(dwt3,rest_s2)
print("\nReconstructed Signal A2:", rest_s2)

rest_s1 = np.empty(2*len(approximation(dwt2)))
rest_s1 = inverse_wavelet(dwt2,rest_s1)
print("\nReconstructed Signal A1:", rest_s1)

rest_s = np.empty(2*len(approximation(dwt)))
rest_s = inverse_wavelet(dwt,rest_s)
print("\nReconstructed Signal:", rest_s)
print("Original Signal:", s)


Approximation coefficients (A3): [79.19595949]

Reconstructed Signal A2: [44. 68.]

Reconstructed Signal A1: [45.254834   16.97056275 28.28427125 67.88225099]

Reconstructed Signal: [32. 32. 16.  8. 24. 16. 64. 32.]
Original Signal: [32 32 16  8 24 16 64 32]


# 2 dimensions

## Decomposition

In [11]:
def dividir_matriz(matriz): # divide the matrix into 4 quadrants
    filas, columnas = matriz.shape
    mitad_filas = filas // 2
    mitad_columnas = columnas // 2
    submatrices = [matriz[i:i+mitad_filas, j:j+mitad_columnas] for i in range(0, filas, mitad_filas) 
                   for j in range(0, columnas, mitad_columnas)]
    return submatrices

def direct_wavelet2D_H(matrix): # does the direct wavelet transform horizontally
    Nf,output = matrix.shape[0], np.zeros_like(matrix,dtype=float)
    for c in range(0, Nf, 2):
        output[:, c//2] = (matrix[:, c] + matrix[:, c+1]) / root2 
        output[:, 2 if c == 0 else 3 if c == 2 else c] = (matrix[:, c] - matrix[:, c+1]) / root2
    return output

def direct_wavelet2D_V(matrix): # does the direct wavelet transform vertically
    Nf,Nc,output = matrix.shape[0], matrix.shape[0]//2, np.copy(matrix)
    for f in range(0, Nf, 2):
        output[f // 2, :Nc] = (matrix[f, :Nc] + matrix[f + 1, :Nc]) / root2
        output[2 if f == 0 else 3 if f == 2 else f, :Nc] = (matrix[f, :Nc] - matrix[f + 1, :Nc]) / root2
    return output

## 0 2

In [17]:
matriz = np.array([[32, 8, 4, 2],
                   [16, 0, 2, 4],
                   [8, 4, 8, 0],
                   [64, 2, 8, 16]])

# matriz = np.random.randint(low=1, high=101, size=(6, 6)) * 2


# DIRECT WAVELET TRANSFORM
result = direct_wavelet2D_V(direct_wavelet2D_H(matriz))
# result = direct_wavelet2D_V(matriz)

print("\nOriginal Signal:\n", matriz)
print("\nDirect wevelet Matrix\n",result)

cA, cV, cH, cD = dividir_matriz(result)

# print("\nApproximation coefficients (A1):\n", cA)
# print("\nDetail coefficients (D1)")
# print(f"Horizontal Details (cH):\n{cH}\n")
# print(f"Vertical Details (cV):\n{cV}\n")
# print(f"Diagonal Details (cD):\n{cD}\n")


Original Signal:
 [[32  8  4  2]
 [16  0  2  4]
 [ 8  4  8  0]
 [64  2  8 16]]

Direct wevelet Matrix
 [[ 28.           6.          16.97056275   1.41421356]
 [ 39.          16.          11.3137085   -1.41421356]
 [ 12.           0.           2.82842712   5.65685425]
 [-27.          -8.          43.84062043  -5.65685425]]


## Reconstriction

In [ ]:
# INVERSE WAVELET TRANSFORM
# reconstructed_matriz = np.zeros_like(result, dtype=float)
# reconstructed_matriz = inverse_wavelet2D_H(inverse_wavelet2D_V(result))
# reconstructed_matriz = (inverse_wavelet2D_V(result))
# print("\nReconstructed Signal:\n", reconstructed_matriz)

In [13]:
def haar_wavelet_transform(matrix):
    n = int(matrix.shape[0])
    n_half = int(n /2)
    transformed_matrix = np.zeros((n,n))

    if(n >= 2):
        for i in range(0, n):
            aux = 0
            aux2 = int(n_half + aux)
            for j in range(0, n, 2):

                sum = (matrix[i, j] + matrix[i, j+1]) / root2
                dif = (matrix[i, j] - matrix[i, j+1]) / root2
                transformed_matrix[i,aux] = sum
                transformed_matrix[i,aux2] = dif
                aux = aux + 1
                aux2 = aux2 + 1
        transformed_matrix2 = np.copy(transformed_matrix)
        for j in range(0, n_half):
            aux = 0
            aux2 = int(n_half + aux)
            for i in range(0, n, 2):

                sum = (transformed_matrix[i, j] + transformed_matrix[i+1, j]) / root2
                dif = (transformed_matrix[i, j] - transformed_matrix[i+1, j]) / root2

                transformed_matrix2[aux,j] = sum
                transformed_matrix2[aux2,j] = dif
                aux = aux + 1
                aux2 = aux2 + 1

        transformed_matrix3 = transformed_matrix2[:n_half,:n_half]

    return transformed_matrix2, transformed_matrix3

image_result, image_aprox = haar_wavelet_transform(matriz)

print(image_result)

[[179.         137.         211.          11.3137085  108.8944443
   -7.07106781]
 [331.         209.         205.          66.46803743   8.48528137
  -39.59797975]
 [160.         173.         272.         -16.97056275   0.
  104.65180362]
 [ 41.          25.           7.          21.21320344 -52.32590181
   24.04163056]
 [-19.         -69.         -33.         -96.16652224 -59.39696962
  -18.38477631]
 [-20.          31.          -2.          39.59797975  60.81118318
  -21.21320344]]


In [14]:
# def inverse_wavelet2D_V(output):
#     N, N2, signal = output.shape[0], output.shape[0] // 2,np.zeros_like(output,dtype=float)
#     for i in range(0, N-1, 2):
#         signal[i, :] = (output[i//2, :] + output[i//2 + N2, :]) / root2
#         signal[i+1, :] = (output[i//2, :] - output[i//2 + N2, :]) / root2
#     return signal

# def inverse_wavelet2D_H(output):
#     N, N2, signal = output.shape[1], output.shape[1] // 2, np.zeros_like(output,dtype=float)
#     for i in range(0, N-1, 2):
#         signal[ :,i] = (output[ :,i//2] + output[ :,i//2 + N2]) / root2
#         signal[ :,i+1] = (output[ :,i//2] - output[ :,i//2 + N2]) / root2
#     # return signal
#     return np.round(signal, decimals=2)

## Example with an Image

In [15]:
import cv2
import IPython
import matplotlib.pyplot as plt


def imshow(img):
    _,ret = cv2.imencode('.jpg', img) 
    i = IPython.display.Image(data=ret)
    IPython.display.display(i)

In [16]:
imgOr = cv2.imread(('image_color.jpg'), cv2.IMREAD_GRAYSCALE)
imgOr = cv2.resize(imgOr,(64,64))
imgOr = np.array(imgOr)

# DIRECT WAVELET TRANSFORM 
# result = direct_wavelet2D_H(direct_wavelet2D_V(imgOr))
result = haar_wavelet_transform(imgOr)

cA, cV, cH, cD = dividir_matriz(result)

reconstructed = np.zeros_like(imgOr, dtype=float)
reconstructed = inverse_wavelet2D_H(inverse_wavelet2D_V(result))

fig, axes = plt.subplots(1, 3, figsize=(10, 10))
axes[0].imshow(imgOr, cmap='gray')
axes[0].set_title('Original')
axes[1].imshow(reconstructed, cmap='gray')
axes[1].set_title('Reconstructed')
axes[2].imshow(result, cmap='gray')
axes[2].set_title('Result')

plt.tight_layout()
plt.show()

fig, axes = plt.subplots(1, 4, figsize=(10, 10))
axes[0].imshow(cA, cmap='gray')
axes[0].set_title('Aproximación')
axes[1].imshow(cH, cmap='gray')
axes[1].set_title('Horizontal')
axes[2].imshow(cV, cmap='gray')
axes[2].set_title('Vertical')
axes[3].imshow(cD, cmap='gray')
axes[3].set_title('Diagonal')

plt.tight_layout()
plt.show()

C:\Users\ZIGH\AppData\Local\Temp\ipykernel_9780\2425618.py:13: RuntimeWarning: overflow encountered in ubyte_scalars
  dif = (matrix[i, j] - matrix[i, j+1]) / root2
C:\Users\ZIGH\AppData\Local\Temp\ipykernel_9780\2425618.py:12: RuntimeWarning: overflow encountered in ubyte_scalars
  sum = (matrix[i, j] + matrix[i, j+1]) / root2


AttributeError: 'tuple' object has no attribute 'shape'

In [ ]:
print("Imagen original:\n", imgOr)
print("\nDir Wavelet:\n", direct_wavelet2D_H(imgOr))
print("\nInv Wavelet:\n", inverse_wavelet2D_H(direct_wavelet2D_H(imgOr)))

Imagen original:
 [[107 111 110 ... 109 117 120]
 [121 121 101 ... 135 139 141]
 [116 115 117 ... 122 120 115]
 ...
 [102 109 110 ...  94  89  70]
 [ 87  87 104 ...  82  98  91]
 [ 71  69  81 ...  45  47  58]]

Dir Wavelet:
 [[154 148 169 ... 109 117 120]
 [171 153 173 ... 135 139 141]
 [163 158 164 ... 122 120 115]
 ...
 [149 142 164 ...  94  89  70]
 [123 132 141 ...  82  98  91]
 [ 98 125 111 ...  45  47  58]]

Inv Wavelet:
 [[  2.12  34.65   0.   ...  29.7   21.92  33.23]
 [ 25.46  35.36   1.41 ...  89.8  111.02  92.63]
 [ 10.61  38.89  12.73 ...  41.01  17.68  36.06]
 ...
 [142.13  68.59 128.69 ...  25.46 128.69  29.7 ]
 [116.67  57.28 120.21 ...  16.97 158.39  29.7 ]
 [ 98.99  39.6  115.26 ...  13.44  93.34  11.31]]
